# Sudden Approximation for the Particle-in-a-Box, Revisited and Extended

## &#x1f469;&#x1f3fd;&#x200d;&#x1f4bb; Assignment: Complete the Below Notebook
1. Complete the code blocks indicated by  `### START YOUR CODE HERE`  and  `### END YOUR CODE HERE` .
1. Upload your notebook *with the same name* and confirm its correctness. The marking scheme is provided at the end of this document.
1. There are *two* exercises and one bonus exercise in this notebook. Both exercises have two parts. The other portions of the notebook are explanation and utilities to help you with the exercises.

## The Sudden Approximation
As an introduction to time-dependent phenomena, we considred a case where the Hamiltonian changed abruptly. For example,
$$
\hat{H} = 
\begin{cases}
\hat{H}_a & t \le 0 \\
\hat{H}_b & t > 0
\end{cases}
$$

Of course, in reality, a system never changes infinitely quickly. However, there are cases where the change in the Hamiltonian associated with a system is much faster than the relaxation time of the system. A typical example is radioactive decay, wherein the identity of an atomic nucleus changes on a timescale that is, from the viewpoint of the electrons in the system, nearly instantaneous. 

Suppose we have solved the Schr&ouml;dinger equation for the initial and final Hamiltonians,
$$
\begin{align}
\hat{H}_a | \psi_{a,n} \rangle &= E_{a,n} | \psi_{a,n} \rangle \\
\hat{H}_b | \psi_{b,n} \rangle &= E_{b,n} | \psi_{b,n} \rangle
\end{align}
$$

Let's assume that the system is originally in a stationary state, described by the quantum number $n=n_0$. Because the Hamiltonian is time-dependent, we need to consider the time-dependent wavefunctions. [Recall](https://qchem.qc-edu.org/SchrodingerEq.html#the-time-independent-schrodinger-equation-tise) that the solution to the (time-dependent) Schr&ouml;dinger equation for a Hamiltonian that is not changing in time has the form
$$
| \Psi_{a;n_0}(t) \rangle = \left| \psi_{a,n_0}e^{\tfrac{-iE_{a,n_0}t}{\hbar}} \right\rangle
$$
This is the wavefunction for $t<0$. The Hamiltonian for $t>0$ also is not changing, and so the Schr&ouml;dinger equation for $t>0$ has a solution like
$$
| \Psi_b(t) \rangle = \sum_{n} c_n | \Psi_{b;n}(t) \rangle
$$
where
$$
| \Psi_{b;n}(t) \rangle = \left| \psi_{b,n}e^{\tfrac{-iE_{b,n}t}{\hbar}} \right\rangle
$$
We don't know what the coefficients $c_n$ are, however. Yet we do know that the wavefunction at $t=0$ can be expressed in two different ways, ergo:
$$
|\psi_{a,n_0} \rangle = \sum_{n} c_n | \psi_{b,n} \rangle
$$
The time dependence drops out because $e^{\tfrac{-iE\cdot 0}{\hbar}}=e^0=1$. 

To evaluate the expansion coefficients $c_n$, multipy both sides of the equation by an eigenfunction of $\hat{H}_b$ and multiply, obtaining:
$$
\begin{align}
\langle \psi_{b,m} | \psi_{a,n_0} \rangle &= \left\langle \psi_{b,m}^* \bigg| \sum_n c_n \psi_{b,n} \right\rangle \\
\langle \psi_{b,m} | \psi_{a,n_0} \rangle &= \sum_n c_n \langle \psi_{b,m}^*(x) | \psi_{b,n}\rangle \\
\langle \psi_{b,m} | \psi_{a,n_0} \rangle &= c_m
\end{align}
$$
The second line uses the fact that the integral of a sum is the sum of the integrals, while the third line uses the fact that eigenfunctions of the time-independent Schr&ouml;dinger equation are orthogonal and normalized,
$$
\begin{align}
\langle \psi_{b,m}^*(x) | \psi_{b,n}(x) \rangle &=
\begin{cases}
1 &m=n \\
0 &m\ne n
\end{cases} \\
&= \delta_{mn}
\end{align}
$$

*We explored the sudden approximation for moving the rightmost boundary of the 1-dimensional particle-in-a-box in a previous exercise. These exercises extend that treatment to two electrons and to a 1-dimensional box where both boundaries more. We will use atomic units throughout.*

**The following code initializes useful libraries and provides some useful reference functions.**
 - the wavefunction of a one-dimensional particle-in-a-box.
 - the energy of a one-dimensional particle-in-a-box.
 - the expansion coefficients for a one-dimensional particle-in-a-suddenly-changed-box.
 - the probability of observing an electron at position $x$ at time $t$ for a particle-in-a-suddenly-changed-box. 

In [27]:
# Some preliminaries follow.
import numpy as np          
from scipy.integrate import quad
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Use the function we've defined previously for the wavefunction
def compute_wavefunction(x, n, a):
    """Compute 1-dimensional particle-in-a-box wave-function value(s).
    
    Parameters
    ----------
    x: float or np.ndarray
        Position of the particle.
    n: int
        Quantum number value.
    a: float 
        Length of the box.
    """
    # check argument n
    if not (isinstance(n, int) and n > 0):
        raise ValueError("Principle quantum number should be a positive integer.")
    # check argument a
    if a <= 0.0:
        raise ValueError("Box length should be positive.")
    # check argument x
    if not (isinstance(x, float) or hasattr(x, "__iter__")):
        raise ValueError("Position should be a float or an array.")
        
    # compute wave-function
    value = np.sqrt(2 / a) * np.sin(n * np.pi * x / a)

    # set wave-function values out of the box equal to zero
    if hasattr(x, "__iter__"):
        value[x > a] = 0.0
        value[x < 0] = 0.0
    else:
        if x < 0.0 or x > a:
            value = 0.0
    return value

# Define a function for the energy of a particle in a box
# with length a and quantum number n [in atomic units!]
# The length is input in Bohr (atomic units)
def compute_energy(n, a):
    "Compute 1-dimensional particle-in-a-box energy."
    return n**2 * np.pi**2 / (2 * a**2)

def compute_amplitude_1D(n0, m, a, b):
    """Compute probability amplitude for state n_0 -> m, when box length a -> b with sudden approx.
    Compute the wavefunction coefficient (probability amplitude) of the m-th state for a 
    1-dimensional particle-in-a-box that starts in quantum state n0 with length a, and 
    changes to length b. The sudden approximation is used, and the amplitude is evaluated 
    analytically.

    Parameters
    ----------
    n0: int
        label of initial state.
    a: float 
        Initial length of the box.
    m: int
        label of final state.
    b: float
       Final length of the box.
    """
    # check argument n0
    if not (isinstance(n0, int) and n0 > 0):
        raise ValueError("Initial principle quantum number n0 should be a positive integer.")
    # check argument m
    if not (isinstance(m, int) and m > 0):
        raise ValueError("Final principle quantum number m should be a positive integer.")
    # check argument a
    if a <= 0.0:
        raise ValueError("Initial box length, a, should be positive.")
    # check argument b
    if b <= 0.0:
        raise ValueError("Final box length, b, should be positive.")
    
    # Evaluate the probability amplitude, c_m, using the exact equation.
    ### START YOUR CODE HERE
    # Compute the upper limit of the integral.
    L = min(a,b)
    # Compute the two complicated terms involving trig functions.
    if (np.abs(m*a-n0*b) > 1e-12):
        #avoid near-zero denominator
        term1 = np.sin((m*a-n0*b)*np.pi*L/(a*b)) / ((m*a-n0*b)*np.pi)
    else:
        #when denominator is near-zero, sin(k*denom)/denom .approx. k 
        term1 = L/(a*b)
        
    term2 = np.sin((m*a+n0*b)*np.pi*L/(a*b)) / ((m*a+n0*b)*np.pi)
    # Put it all together to compute the probability amplitude
    c_m = np.sqrt(a*b)*(term1 - term2)
    ### END YOUR CODE HERE
    return c_m 

def compute_probability_1D(n0, a, b, x, t):
    """Compute probability of observing particle at x at time t.
    Compute the probability of observing a particle at position
    x at time t. Up until t=0, the particle is in the
    stationary state n0 of a box with length a. At time 
    t=0, the length of the box suddenly changes to b.

    Parameters
    ----------
    n0: int
        quantum number of initial stationary state.
    a: float 
        Initial length of the box.
    b: float
       Final length of the box.
    t: float or np.ndarray
       time. Assumed nonnegative since that is where
       interesting things happen.
    x: float or np.ndarray
        Position of the particle.
    """

    # Most variables are fully checked in lower-level routines
    # but we should still make sure the time is nonnegative:
    # check type of time variable
    if not (isinstance(t, float) or hasattr(t, "__iter__")):
        raise ValueError("Time should be a float or an array.")
    if np.min(t) < 0.0:
        raise ValueError("The implemented equations do not hold for time < 0.")
    
    # The coefficients decay roughly as 1/m^2. By default
    # we compute up to m=100, but one may wish to increase
    # (more accurate) or decrease (faster) this number
    m_max = 200

    ### START YOUR CODE HERE
    #compute wavefunction for x and t
    wavefunction = 0
    for m in range(1,m_max+1):
        wavefunction = wavefunction + ( compute_amplitude_1D(n0, m, a, b)
        * np.exp(-1j*compute_energy(m, b)*t)
        * compute_wavefunction(x,m,b) )

    # compute probability
    prob = np.abs(wavefunction)**2
    
    ### END YOUR CODE HERE

    # set values out of the box equal to zero
    if hasattr(x, "__iter__"):
        prob[x > b] = 0.0
        prob[x < 0] = 0.0
    else:
        if x < 0.0 or x > b:
            prob = 0.0

    return prob

## Sudden Approximation for the 2-dimensional Particle-in-a-Box

Consider the special cases of a sudden change in a two-dimensional electron-in-a-box. The two Hamiltonians are (in atomic units), for $t \le 0$
$$
\hat{H}_{t\le0} = -\tfrac{1}{2} \tfrac{d^2}{dx^2} -\tfrac{1}{2} \tfrac{d^2}{dy^2} + V_{a_x}(x) + V_{a_y}(y) \\
\hat{H}_{t>0} = -\tfrac{1}{2} \tfrac{d^2}{dx^2} -\tfrac{1}{2} \tfrac{d^2}{dy^2} + V_{b_x}(x) + V_{b_y}(y)
$$
where
$$
V_a(x) = 
\begin{cases}
+\infty && x \le 0 \\
0 && 0 < x < a \\
+\infty && a \le x
\end{cases}
$$


## &#x1f469;&#x1f3fd;&#x200d;&#x1f4bb; **Exercise 1.** Apply the sudden approximation to the 2-dimensional particle-in-a-box.

### &#x1f469;&#x1f3fd;&#x200d;&#x1f4bb; **Exercise 1a.** Write a function to compute the probability amplitudes, $c_{m_x,m_y}$, for a sudden change in box size. 
I.e., the final wavefunction is expanded in the basis of solutions for the final Hamiltonian,
$$
| \Psi_b(t) \rangle = \sum_{m_x,m_y} c_{m_x,m_y} | \Psi_{b;m_x,m_y}(t) \rangle
$$ 

### &#x1f469;&#x1f3fd;&#x200d;&#x1f4bb; **Exercise 1b.** Write a function to compute the probability of observing the particle at position $(x,y)$ at time $t$.

In [28]:
# Exercise 1a. Evaluate the coefficients of expansion for the probability amplitude.
def compute_amplitude_2D(n0x, n0y, mx, my, ax, ay, bx, by):
    """Compute probability amplitude for state n_0x,n_0y -> mx,my, when box size suddenly changes.
    Compute the wavefunction coefficient (probability amplitude) of the (mx,my)-th state for a 
    2-dimensional particle-in-a-box that starts in quantum state n0x,n0y with 0<x<ax and 0<y<ay, and 
    changes to 0<x<bx and 0<y<by. The sudden approximation is used.

    Parameters
    ----------
    n0x, n0y: int
        quantum numbers of the initial state.
    ax,ay: float 
        The initial box is defined by 0<x<ax and 0<y<ay.
    mx,my: int
        label of final quantum state.
    bx,by: float
       The final box is defined by 0<x<bx and 0<y<by.
    """
    # check arguments n0x and n0y
    if not (isinstance(n0x, int) and n0x > 0):
        raise ValueError("Initial principle quantum number n0x should be a positive integer.")
    if not (isinstance(n0y, int) and n0y > 0):
        raise ValueError("Initial principle quantum number n0y should be a positive integer.")
    # check arguments mx and my
    if not (isinstance(mx, int) and mx > 0):
        raise ValueError("Final principle quantum number mx should be a positive integer.")
    if not (isinstance(my, int) and my > 0):
        raise ValueError("Final principle quantum number my should be a positive integer.")
    # check argumentx ax and ay
    if ax <= 0.0:
        raise ValueError("Initial box length, ax, should be positive.")
    if ay <= 0.0:
        raise ValueError("Initial box length, ay, should be positive.")
    # check argument bx and by
    if bx <= 0.0:
        raise ValueError("Final box length, bx, should be positive.")
    if by <= 0.0:
        raise ValueError("Final box length, by, should be positive.")
    
    ### START YOUR CODE HERE

    ### END YOUR CODE HERE

In [29]:
# Exercise 1b. Evaluate the probability of observing an electron at position (x,y) at time t.
def compute_probability_2D(n0x, n0y, ax, ay, bx, by, x, y, t):
    """Compute probability of observing an electron at position (x,y) at time t.
    Compute the probability of observing an electron at position (x,y) at time t.
    The initial box is defined by 0<x<ax and 0<y<ay. The final box is defined by 0<x<bx and 0<y<by.
    The particle initial state of the particle is the stationary state with quantum numbers (n0x,n0y).

    Parameters
    ----------
    n0x, n0y: int
        quantum numbers of the initial state.
    ax,ay: float 
        The initial box is defined by 0<x<ax and 0<y<ay.
    bx,by: float
       The final box is defined by 0<x<bx and 0<y<by.
    x,y: float
       Position of the electron.
    t: float
       Time.
    """
    # Most variables are fully checked in lower-level routines
    # but we should still make sure the time is nonnegative:
    # check type of time variable
    if not (isinstance(t, float) or hasattr(t, "__iter__")):
        raise ValueError("Time should be a float or an array.")
    if np.min(t) < 0.0:
        raise ValueError("The implemented equations do not hold for time < 0.")
    
    ### START YOUR CODE HERE
    #compute probability


    ### END YOUR CODE HERE

print("Starting from ground state, the probability amplitude of finding the particle lowest-accessible state of a doubled box is:", compute_amplitude_2D(1, 1, 2, 2, 1.0, 1.0, 2.0, 2.0))
print("Starting from ground state, the probability of finding the particle at the midpoint of a doubled box at t=0.5 is:", compute_probability_2D(1, 1, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 0.5))

Starting from ground state, the probability amplitude of finding the particle lowest-accessible state of a doubled box is: 0.5000000000000001
Starting from ground state, the probability of finding the particle at the midpoint of a doubled box at t=0.5 is: 0.19766250752776565


## Sudden Approximation for Changing Both Boundaries of a 1-dimensional Particle-in-a-Box

Consider the special cases of a sudden change in *both* boundaries of a one-dimensional electron-in-a-box. The two Hamiltonians are (in atomic units), for $t \le 0$
$$
\hat{H}_{t\le0} = -\tfrac{1}{2} \tfrac{d^2}{dx^2} + V_{a_0,a_1}(x)\\
\hat{H}_{t>0} = -\tfrac{1}{2} \tfrac{d^2}{dx^2} + V_{0,b_1}(x)
$$
where
$$
V_{a_0,a_1}(x) = 
\begin{cases}
+\infty && x \le a_0 \\
0 && a_0 < x < a_1 \\
+\infty && a_1 \le x
\end{cases}
$$

For simplicity, we choose the final box to begin at the origin.

## &#x1f469;&#x1f3fd;&#x200d;&#x1f4bb; **Exercise 2.** Apply the sudden approximation to the 1-dimensional particle-in-a-box where both boundaries shift.

### &#x1f469;&#x1f3fd;&#x200d;&#x1f4bb; **Exercise 2a.** Write a function to compute the probability amplitudes, $c_m$, for a sudden change in box size/position. 

### &#x1f469;&#x1f3fd;&#x200d;&#x1f4bb; **Exercise 2b.** Write a function to compute the probability of observing the particle at position $x$ at time $t$.

In [30]:
# Exercise 2a. Finish the function for computing the probability amplitude for the m-th state, assuming
#              the initial state is n0 with a box a0 < x < a1 and the final state is in a box with 0 < x < b1.
def compute_amplitude(n0, m, a0, a1, b1):
    """Compute probability amplitude for state n_0 -> m, when box suddenly changes in size/position.
    Compute the wavefunction coefficient (probability amplitude) of the m-th state for a 
    1-dimensional particle-in-a-box that starts in quantum state n0 with a0 < x < a1
    changes to 0 < x < b1. The sudden approximation is used, and the amplitude is evaluated 
    numerically.

    Parameters
    ----------
    n0: int
        label of initial state.
    a0: float 
        Initial start position of the box.
    a1: float 
        Initial start position of the box.
    m: int
        label of final state.
    b1: float
       Final end position (and length) of the box, which is assumed to start at x=0.
    """
    
    # check arguments n0 and m
    if not (isinstance(n0, int) and n0 > 0):
        raise ValueError("Initial principle quantum number n0 should be a positive integer.")
    if not (isinstance(m, int) and m > 0):
        raise ValueError("Final principle quantum number m should be a positive integer.")

    # check arguments a0, a1, b1
    if not isinstance(a0, float):
        raise ValueError("Start position of initial box, a0, should be a float.")
    if not isinstance(a1, float):
        raise ValueError("End position of initial box, a1, should be a float.")
    if not isinstance(b1, float):
        raise ValueError("End position of final box, b1, should be a float.")
    if (a1 <= a0):
        raise ValueError("End position of initial box, a1, should be larger than start position of initial box, a0.")
    if (b1 <= 0):
        raise ValueError("End position of final box, b1, should be positive. (The box starts at zero by assumption.)")

    #Define the lengths of the initial and final boxes.
    a = a1 - a0
    b = b1

    # Compute the probability amplitude, c_m
    ### BEGIN YOUR CODE HERE
    
    ### END YOUR CODE HERE

#Check by referencing the previous function
print("The amplitude for m=6 when n0=2 and the box-size triples",compute_amplitude(2,6,0.0,1.0,3.0))
print("The amplitude for m=6 when n0=2 and the box-size triples",compute_amplitude_1D(2,6,1.0,3.0))

The amplitude for m=6 when n0=2 and the box-size triples 0.5773502691896258
The amplitude for m=6 when n0=2 and the box-size triples 0.5773502691896257


In [31]:
# Exercise 2b. Finish the function for computing the probability of observing an electron at position x at time t,
#              assuming the initial state is n0 with a box a0 < x < a1 and the final box has 0 < x < b1.
def compute_probability(n0, a0, a1, b1, x, t):
    """Compute probability of observing an electron at position x at a time t when box suddenly changes size/position.
    Compute the probability of observing an electon at position x and time t for a
    1-dimensional particle-in-a-box that starts in quantum state n0 with a0 < x < a1
    (for t<0) and suddenly changes to 0 < x < b1 at time t=0.

    Parameters
    ----------
    n0: int
        label of initial state.
    a0: float 
        Initial start position of the box.
    a1: float 
        Initial start position of the box.
    b1: float
       Final end position (and length) of the box, which is assumed to start at x=0.
    x: float
        Position of the electron
    t: float
        time of observation, t>=0

    Returns
    -------
    float
        Probability of observing an electron at position x at time t.
    """
    # Most variables are fully checked in lower-level routines
    # but we should still make sure the time is nonnegative:
    # check type and sign of time variable
    if not (isinstance(t, float) or hasattr(t, "__iter__")):
        raise ValueError("Time should be a float or an array.")
    if np.min(t) < 0.0:
        raise ValueError("The implemented equations do not hold for time < 0.")
    
    # The coefficients decay roughly as 1/m^2. By default
    # we compute up to m=100, but one may wish to increase
    # (more accurate) or decrease (faster) this number
    m_max = 300
    
    #Compute probability variable, which I've called "prob" in my code.
    ### START YOUR CODE HERE
    #compute wavefunction for x and t
    
    ### END YOUR CODE HERE
    
    # set values out of the box equal to zero
    if hasattr(x, "__iter__"):
        prob[x > b1] = 0.0
        prob[x < 0] = 0.0
    else:
        if x < 0.0 or x > b1:
            prob = 0.0

    return prob

#Check the probability by referencing the previous function
print("The probability at x=1/2 and t=1/2 when n0=2 and the box-size triples",compute_probability(2,0.0,1.0,3.0,0.5,0.5))
print("The probability at x=1/2 and t=1/2 when n0=2 and the box-size triples",compute_probability_1D(2,1.0,3.0,0.5,0.5))        

The probability at x=1/2 and t=1/2 when n0=2 and the box-size triples 0.07002471332358527
The probability at x=1/2 and t=1/2 when n0=2 and the box-size triples 0.07007078578461852


### Visualization
The following visualization shows what happens when the box does not change size, but merely shifts over by half its width. 

The code blocks below make movies, but they take a pretty long time to run and don't work on every platform. But the following movies are elucidative: 

$n_0 = 1$
![](suddengif_n0_eq_1.gif)

$n_0 = 2$
![](suddengif_n0_eq_2.gif)

In [34]:
from IPython import display
from IPython.display import HTML

# Plot the probability as a function of time.

# Choose the initial (a0,a1) and final (b1) size/position of the box.
b1 = 1.0
a0 = -0.5*b1
a1 = 0.5*b1
# Choose the initial quantum state
n0 = 1
# Choose the length of time for the simulation
time_length = .1

#Choose how many x-points you wish to plot
xpoints = 100    #good option for decent plot
                #increase this number to make a smoother plot
                #xpoints = 200 gives a pretty smooth plot.
xstep = b1/xpoints

#Choose the timestep (in atomic units) and number of frames (timesteps)
time_step = 0.00005    #good option for decent plot. 
                    #reduce this number to make a smoother plot.
nframes = int(time_length/time_step)

fig, ax = plt.subplots()

#This plots the probability at time=0
x = np.arange(0, b1, xstep)
line, = ax.plot(x, compute_probability(n0,a0,a1,b1,x,0.0))

#This computes the probability at time i*tstep
def animate(i):
    line.set_ydata(compute_probability(n0,a0,a1,b1,x,i*time_step))  # update the data.
    return line,

ani = animation.FuncAnimation(
    fig, animate, frames=range(0,nframes), blit=False, repeat=True, interval=10)

plt.close(ani._fig)


ani.save("suddengif_n0_eq_1.gif") #saves file as gif; optional. Usually works out-of-the-box.
![](suddengif_n0_eq_1.gif)
#ani.save("sudden.mp4") #saves file as mp4; optional; requires ffmpeg
HTML(ani.to_html5_video())   #This works well for Google Colab or whenever ffmpeg is available.

'[]' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# Choose the initial (a0,a1) and final (b1) size/position of the box.
b1 = 1.0
a0 = -0.5*b1
a1 = 0.5*b1
# Choose the initial quantum state
n0 = 2
# Choose the length of time for the simulation
time_length = 10

#Choose how many x-points you wish to plot
xpoints = 100    #good option for decent plot
                #increase this number to make a smoother plot
                #xpoints = 200 gives a pretty smooth plot.
xstep = b1/xpoints

#Choose the timestep (in atomic units) and number of frames (timesteps)
time_step = 0.01    #good option for decent plot. 
                    #reduce this number to make a smoother plot.
nframes = int(time_length/time_step)

fig, ax = plt.subplots()

#This plots the probability at time=0
x = np.arange(0, b1, xstep)
line, = ax.plot(x, compute_probability(n0,a0,a1,b1,x,0.0))

#This computes the probability at time i*tstep
def animate(i):
    line.set_ydata(compute_probability(n0,a0,a1,b1,x,i*time_step))  # update the data.
    return line,

ani = animation.FuncAnimation(
    fig, animate, frames=range(0,nframes), blit=False, repeat=True, interval=10)

plt.close(ani._fig)

ani.save("suddengif_n0_eq_2.gif") #saves file as gif; optional. Usually works out-of-the-box.
![](suddengif_n0_eq_2.gif)
#ani.save("sudden.mp4") #saves file as mp4; optional; requires ffmpeg
HTML(ani.to_html5_video())   #This works well for Google Colab or whenever ffmpeg is available.

'[]' is not recognized as an internal or external command,
operable program or batch file.


## &#x2696;&#xfe0f; Marking Scheme
&#x2611;&#xfe0f; Successful completion of the notebook, together with the ability to discuss your strategy, earns an **S**. If you complete Exercise 1 *or* Exercise 2, and discuss your solution, then you can earn an S-. 

&#x1f4b0; For an **S+**, Answer the following two questions in a markdown cell at the end of the notebook.
- Can you think of examples where the particle-in-a-box is in an eigenstate even for $t>0$? Give two examples in both 1 and 2 dimensions.
- Can you think of an example where the particle-in-a-box wavefunction is zero for $t>0$? Such a process is 100% forbidden (to the level of validity of the sudden approximation). Does this make sense and, if so, why?